# Google Colab 데이터 로드

In [ ]:
#Step 1. 구글 코랩에 한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126102 files and direc

In [2]:
#Step 2.분석할 데이터가 저장된 파일을 불러와서 변수에 할당합니다.
from google.colab import files
myfile = files.upload()
import io
import pandas as pd
#pd.read_csv로 csv파일 불러오기
wine = pd.read_csv(io.BytesIO(myfile['와인.csv']),
                       encoding='cp949')
wine

Saving 와인.csv to 와인.csv


,알콜도수,당도,산도_ph,종류
0,9.4,1.9,3.51,레드와인
1,9.8,2.6,3.20,레드와인
2,9.8,2.3,3.26,레드와인
3,9.8,1.9,3.16,레드와인
4,9.4,1.9,3.51,레드와인
...,...,...,...,...
6492,11.2,1.6,3.27,화이트와인
6493,9.6,8.0,3.15,화이트와인
6494,9.4,1.2,2.99,화이트와인
6495,12.8,1.1,3.34,화이트와인


# 로컬 데이터 로드

In [ ]:
import pandas as pd

wine = pd.read_csv('와인.csv', encoding='cp949')
wine

,알콜도수,당도,산도_ph,종류
0,9.4,1.9,3.51,레드와인
1,9.8,2.6,3.20,레드와인
2,9.8,2.3,3.26,레드와인
3,9.8,1.9,3.16,레드와인
4,9.4,1.9,3.51,레드와인
...,...,...,...,...
6492,11.2,1.6,3.27,화이트와인
6493,9.6,8.0,3.15,화이트와인
6494,9.4,1.2,2.99,화이트와인
6495,12.8,1.1,3.34,화이트와인


# 공통 실습 코드

In [3]:
from sklearn.model_selection import train_test_split
# 알콜도수, 당도, 산도_ph를 통해 종류 분류
data = wine[['알콜도수','당도','산도_ph']].to_numpy()
target = wine['종류'].to_numpy()

# train, text 데이터 분리
훈련용_data, 테스트용_data, 훈련용_target, 테스트용_target = train_test_split(data,target,test_size=0.2,random_state=40)

In [4]:
#Step 4. GradientBoostiongClassifier 모델 생성
from sklearn.ensemble import GradientBoostingClassifier
# 학습
df = GradientBoostingClassifier()
df.fit(훈련용_data, 훈련용_target)
# score 출력
print(df.score(훈련용_data, 훈련용_target))
print(df.score(테스트용_data, 테스트용_target))

0.8853184529536271
0.8630769230769231


### GradientBoosting 하이퍼 파라미터
- e_estimators : 사용할 트리 개수 (default: 100)
- max_depth : 개별 트리의 깊이 (default: 3)
- learning_rate : 학습률 (default: 0.1)
- min_samples_split : 노드를 분할하기 위한 최소 데이터 수 (default: 2)
- min_samples_leaf : 자식 노드가 가져야할 최소 데이터 수 (default: 1)
- subsample : 학습에 사용하는 샘플의 비율(default: 1)
- loss : 비용함수(CostFunction), (default로 설정)

In [5]:
# 그리드서치 활용하여 하이퍼라파미터 튜닝 - LGB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

# 파라미터 설정
parms = {'max_depth':range(1,10), 'min_samples_split':range(2,20)}

#그리드 서치 객체 생성
gs = GridSearchCV(GradientBoostingClassifier(random_state=50), parms, n_jobs=-1)
# 학습
gs.fit(훈련용_data, 훈련용_target)
# 최적의 하이퍼 파라미터 값 출력
print(gs.best_params_)
# 정확도 출력
df = gs.best_estimator_
print(df.score(훈련용_data, 훈련용_target))
print(df.score(테스트용_data, 테스트용_target))

{'max_depth': 9, 'min_samples_split': 14}
0.9905714835482009
0.8753846153846154
